# 定义问题

* Kaggle描述： It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

预测房价 79个特征的数据集

# 获取数据

Kaggle:https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

# 数据预处理

## 导入环境库

In [ ]:
#load packages， 打印，便于可复现
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time


#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)



## 导入数据模型与可视化库

In [ ]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

## 载入总览数据

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

'''
预测SalePrice的值
'''

# train_df.columns 
# train_df.shape
train_df.describe()
# train_df.info()
# print '%' * 40
# test_df.info()
# train_df.head(10)
# train_df.info()
# print train_df.describe(include = 'all')
# print '%' * 40
# test_df.info()

## 数据4C分析

数据的：

正确性：Reviewing the data, there does not appear to be any aberrant or non-acceptable data inputs.异常值

完整性：NULL / NAN。删除/补全

创造性：Feature engineering is when we use existing features to create new features to determine if they provide new signals to predict our outcome. For this dataset, we will create a title feature to determine if it played a role in survival.

转变：类别数据 -> 独热编码

### 【完整性】 (缺失值)

https://discuss.analyticsvidhya.com/t/what-should-be-the-allowed-percentage-of-missing-values/2456

I：允许的missing values的数目，先定25%

In [ ]:
# # Missing Data的百分比
# total = train_df.isnull().sum().sort_values(ascending=False)
# percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
# missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# # print missing_data.head(20)

# total2 = test_df.isnull().sum().sort_values(ascending=False)
# percent2 = (test_df.isnull().sum()/test_df.isnull().count()).sort_values(ascending=False)
# missing_data2 = pd.concat([total2, percent2], axis=1, keys=['Total', 'Percent'])
# # print missing_data2.head(20)

In [ ]:
train_df.describe(include = 'all')
print ('%' * 40)

# print train_df.loc[:,'MasVnrType']

# train_df['MasVnrType'] = train_df['MasVnrType'].replace(['None'],None )
# train_df.loc[:,'MasVnrType'] =  train_df['MasVnrType'].apply(lambda x: None if x == 'None' else x)
# print type(train_df.loc[1,'MasVnrType'])

limit_missing_values = 0.25
train_limit_missing_values = len(train_df) * limit_missing_values 
print ("Train columns with null values:\n", train_df.columns[train_df.isnull().sum().values > train_limit_missing_values])  # 依列为标准，column
print ('%'*40)

test_limit_missing_values = len(test_df) * limit_missing_values
print ("Test columns with null values:\n", test_df.columns[test_df.isnull().sum().values > test_limit_missing_values])



丢弃过多空数据的列

部分类别数据中的NA非未计数/计数误差，而是无（地下室，游泳池🏊等，填充字符串“None”）

### 确定缺失列

In [ ]:
missing_columns = list(train_df.columns[train_df.isnull().sum() != 0])
print (missing_columns)
# train_df[missing_columns].describe(include = 'all')
print ('%' * 40)
test_missing_columns = list(test_df.columns[test_df.isnull().sum() != 0])
print (test_missing_columns)

In [ ]:
train_missing_numerical = list(train_df[missing_columns].dtypes[train_df[missing_columns].dtypes != 'object'].index)
train_missing_category = [i for i in missing_columns if i not in train_missing_numerical]

test_missing_numerical = list(test_df[test_missing_columns].dtypes[test_df[test_missing_columns].dtypes != 'object'].index)
test_missing_category = [i for i in test_missing_columns if i not in test_missing_numerical]

print ("Train missing numerical: ", train_missing_numerical, "\n")
print ("Train missing category: ", train_missing_category,  "\n")
print ("Test missing numerical: ", test_missing_numerical, "\n")
print ("Test missing category: ", test_missing_category, "\n")

### 类别特征缺失值填充

In [ ]:
# 取众数的特征
train_categories_Mode = ['Electrical']
test_categories_Mode = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'KitchenQual', 'Functional', 'SaleType']

train_categories_none = [i for i in train_missing_category if i not in train_categories_Mode]
test_categories_none = [i for i in test_missing_category if i not in test_categories_Mode]


# 通过字符串“None”填充 
for category in train_categories_none:
    train_df[category].fillna("None", inplace=True)
    
for category in test_categories_none:
    test_df[category].fillna("None", inplace=True)
    
for category in train_categories_Mode:
    train_df[category].fillna(train_df[category].mode()[0], inplace = True)

for category in test_categories_Mode:
    test_df[category].fillna(test_df[category].mode()[0], inplace = True)

### 数值特征缺失值填充

In [ ]:
# 取0的数值特征

for col in ('GarageArea', 'GarageCars', 'GarageYrBlt'):
    train_df[col] = train_df[col].fillna(0)
    test_df[col] = test_df[col].fillna(0)
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    train_df[col] = train_df[col].fillna(0)
    test_df[col] = test_df[col].fillna(0)
    
# 通过中位数填充
for column in train_missing_numerical:
    train_df[column].fillna(train_df[column].median(), inplace=True)

for column in test_missing_numerical:
    test_df[column].fillna(test_df[column].median(), inplace=True) 

In [ ]:
print (train_df.isnull().sum().max())
print (test_df.isnull().sum().max())
# print test_df.loc[test_df.isnull()

# 探索性数据分析 EDA

感谢： https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python 提出的对数据进行探索的方法

### 目标

1、In order to have some discipline in our analysis, we can create an Excel spreadsheet with the following columns:

* <b>Variable</b> - Variable name.
* <b>Type</b> -  There are two possible values for this field: 'numerical' or 'categorical'. 
* <b>Segment</b> - Identification of the variables' segment. We can define three possible segments: building, space or location. When we say 'building', we mean a variable that relates to the physical characteristics of the building (e.g. 'OverallQual'). When we say 'space', we mean a variable that reports space properties of the house (e.g. 'TotalBsmtSF'). Finally, when we say a 'location', we mean a variable that gives information about the place where the house is located (e.g. 'Neighborhood').
* <b>Expectation</b> - Our expectation about the variable influence in 'SalePrice'. We can use a categorical scale with 'High', 'Medium' and 'Low' as possible values.
    
    重要，一一个一个阅读特征描述。是否影响我们购买房屋 -> 如果是，影响重要性？ -> 信息是否早已在其他变量中包含
* <b>Conclusion</b> - Our conclusions about the importance of the variable, after we give a quick look at the data. We can keep with the same categorical scale as in 'Expectation'.
* <b>Comments</b> - Any general comments that occured to us.


2、we can filter the spreadsheet and look carefully to the variables with 'High' 'Expectation'. Then, we can rush into some scatter plots between those variables and 'SalePrice', filling in the 'Conclusion' column which is just the correction of our expectations.

### 目标数据的【身材】 SalePrice （单变量分布）

*Everything started in our Kaggle party, when we were looking for a dance partner. After a while searching in the dance floor, we saw a girl, near the bar, using dance shoes. That's a sign that she's there to dance. We spend much time doing predictive modelling and participating in analytics competitions, so talking with girls is not one of our super powers. Even so, we gave it a try:*

*'Hi, I'm Kaggly! And you? 'SalePrice'? What a beautiful name! You know 'SalePrice', could you give me some data about you? I just developed a model to calculate the probability of a successful relationship between two people. I'd like to apply it to us!'*

In [ ]:
train_df['SalePrice'].describe()

In [ ]:
sns.distplot(train_df['SalePrice'], color='green')

* <b> 正偏态分布 </b> 数据主要集中在10w ~ 25w之间

In [ ]:
print ("偏度为 %f " % train_df['SalePrice'].skew())
print ("峰度为 %f"  % train_df['SalePrice'].kurt())

### 目标数据的Buddies and interests  （两个变量之间的关系）

As soon as 'SalePrice' walked away, we went to Facebook. Yes, now this is getting serious. Notice that this is not stalking. It's just an intense research of an individual, if you know what I mean.

According to her profile, we have some common friends. Besides Chuck Norris, we both know 'GrLivArea' and 'TotalBsmtSF'. Moreover, we also have common interests such as 'OverallQual' and 'YearBuilt'. This looks promising!

To take the most out of our research, we will start by looking carefully at the profiles of our common friends and later we will focus on our common interests.

#### 同数值变量之间的关系 (散点图）

In [ ]:
var = 'BsmtFinSF1' # 房屋居住面积

# concat - Series默认行合并； axis = 1，列合并
data = pd.concat([train_df['SalePrice'], train_df[var]], axis=1) 
# print dataFrame
data.plot.scatter(x=var, y='SalePrice', ylim = (0, 800000)) # y轴限制

房屋价格同居住面积（平方英尺）线性相关

In [ ]:
var = 'TotalBsmtSF'  # 房屋地下室的面积
data = pd.concat([train_df['SalePrice'], train_df[var]], axis=1)
data.plot.scatter(x = var, y = 'SalePrice', ylim = (0, 800000))

房屋价格同地下室的面积成强线性（指数？）相关

存在一些无地下室的房屋，房屋价格分布在200000以内

#### 同类别变量之间的关系 （箱型图）

In [ ]:
var = 'OverallQual'  # 房屋整体材料和光洁度的评估
data = pd.concat([train_df['SalePrice'], train_df[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)  # 横坐标类别，纵坐标目标变量
fig.axis(ymin=0, ymax=800000);

房屋价格同房屋整体材料和光洁度的评估强相关，整体评估分数越高，房屋价格越高

In [ ]:
var = 'YearBuilt'  # 房屋建造年份
data = pd.concat([train_df['SalePrice'], train_df[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))  # 改变大小
fig = sns.boxplot(x=var, y="SalePrice", data=data)  # 横坐标类别，纵坐标目标变量
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);               # x如果都水平放置，会粘到一起。x倾斜90度，减少占的位置

虽然斜率小，但是新房相对于老房来说房屋价格相对较高

#### 总结

1、房屋居住面积和房屋地下室的面积同房屋价格呈正相关

2、房屋整体材料质量和建造年份同房屋价格相关。其中房屋整体材料质量同房屋价格相关性更强

以上：特征选择，主观性

The trick here seems to be the choice of the right features (feature selection) and not the definition of complex relationships between them (feature engineering).

That said, let's separate the wheat from the chaff.

以下：特征工程，客观性

### 客观分析

####  把一些数值特征转化为类别特征

In [ ]:
cols = ['MSSubClass', 'OverallCond', 'BedroomAbvGr', 'YrSold', 'MoSold']

for col in cols:
    train_df[col] = train_df[col].apply(str)
    test_df[col] = test_df[col].apply(str)

* <b>相关性矩阵</b>
* <b>房屋价格相关性矩阵</b>
* <b>最相关变量散点图</b>

#### 整体相关性矩阵

In [ ]:
train_df.columns

In [ ]:
train_df = train_df[['LotFrontage', 'SalePrice', 'MasVnrArea', '1stFlrSF']]

In [ ]:
train_df

In [ ]:
corrmat = train_df.corr()
f, ax = plt.subplots(figsize=(16, 8))
sns.heatmap(corrmat, vmax= .8, square=True);   # vmax 颜色区别，最浅的颜色在0.8

更加宏观

1、“TotalBsmtSF” 同 “1stFlrSF”强相关，表达同样的信息

2、“GarageCars”  同 “GarageArea”强相关，表达同样的信息

3、验证“SalePrice” 同主观选择的特征相关，仍有部分特征需加入"SalePrice"预测

#### 目标变量相关性矩阵 (SalePrice，放大热力图）

In [ ]:
# help(corrmat.nlargest)

In [ ]:
# # cols = corrmat.nlargest(k, 'SalePrice').index
# # print cols
# len (train_df[cols].values)  # 数组，1460 * 10 | 转置 10 * 1460

In [ ]:
old_features = ['1stFlrSF', '2ndFlrSF', 'TotalBsmtSF']
train_df['TotalSF'] = 0
test_df['TotalSF'] = 0
for i in old_features:
    print (train_df['SalePrice'].corr(train_df[i]))
    train_df['TotalSF'] += train_df[i]
    test_df['TotalSF'] += test_df[i]
train_df['SalePrice'].corr(train_df['TotalSF'])

In [ ]:
corrmat = train_df.corr().abs()

In [ ]:
#saleprice correlation matrix
k = 36   #number of variables for heatmap，热力图变量数量 

# nlargest - 根据SalePrice列排序，返回前10个跟SalePrice相关性最高的行
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index 

# cm = corrmat.loc[cols,cols] 同以下cm赋值相同
# 训练集中取出目标列的样本，转置，计算10个特征之间的相关性
cm = np.corrcoef(train_df[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
print (len(train_df.columns))
# print len(drop_columns)
print (len(train_df.dtypes[train_df.dtypes == 'object'].index))



1、确认主观特征选择变量'OverallQual'，'GrLivArea' ，'TotalBsmtSF'同房屋价格强相关

2、因为“GarageCars” 同 “GarageArea”强相关，故选择同房屋价格相关性更高的“GarageCars”

3、同上，选择“TotalBasmtSF”

4、“TotRmsAbvGrd” 同“GrLivArea”强相关，选择“GrLivArea”

5、"YearBuilt"【待】时间序列分析

6、 "FullBath"浴室的品质，从箱型图中可以看出浴室的品质同房屋的价格呈正相关

    同时存在一些无浴室的房屋

#### 最相关变量散点图

In [ ]:
# cols

In [ ]:
# #scatterplot
# sns.set()
# cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
# sns.pairplot(train_df[cols], size = 2.5)
# plt.show();

# 特征分析

### 【正确性】

去除异常值

1、TotalBsmtSF 为0 = 房屋中没有地下室，数据正常

2、故仅针对GrLivArea 做异常值去除

In [ ]:
# olss = []
# numerical_features = ['GrLivArea']
# for feature in numerical_features:
    
#     # 计算25%分位点
#     Q1 = np.percentile(train_df[feature], 25)
    
#     # 计算75%分位点
#     Q3 = np.percentile(train_df[feature], 75)
#     print Q3
    
#     # 异常阶（1.5倍四分位距）IQR
#     step = 1.5 * (Q3 - Q1)
    
#     print "Feature" + feature + "Outlines"
#     print train_df[ (train_df[feature] <= Q1 - step) | (train_df[feature] >= Q3 + step)][numerical_features]
#     ols = features_train[ (train_df[feature] <= Q1 - step) | (train_df[feature] >= Q3 + step)].index.tolist()
#     olss.append(ols)
    
# olss_new = [ii for i in olss for ii in i]
# # print olss_new

# # 列表方法 .count(i) 统计列表中某个元素出现的次数
# more_than_one = list(set([i for i in olss_new if olss_new.count(i) > 1]))
# more_than_two = list(set([i for i in olss_new if olss_new.count(i) > 2]))
# print len(more_than_one), len(more_than_two)

# ## 移除异常点
# # features_train_new = features_train.drop(features_train.index[olss_new]).reset_index(drop = True)
# # labels_train_new = labels_train.drop(features_train.index[olss_new]).reset_index(drop = True)

In [ ]:
var = 'GrLivArea'
data = pd.concat([train_df['SalePrice'], train_df[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [ ]:
# 从散点图中确定去除GrLivArea > 4000 且SalePrice < 200000的点
train_df = train_df.drop(train_df[(train_df['GrLivArea'] > 4000) & (train_df['SalePrice'] < 200000)].index)

### 【创造性】

删除相关性较弱的特征

所有类别特征

In [ ]:
train_df.dtypes[(train_df.dtypes == 'object')].index

In [ ]:
var = 'BldgType'  # 房屋整体材料和光洁度的评估
data = pd.concat([train_df['SalePrice'], train_df[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)  # 横坐标类别，纵坐标目标变量
fig.axis(ymin=0, ymax=800000);

### 【倾斜数据处理】

#### 预测值倾斜，对数处理

* <b> 偏度 </b>  正态分布的偏度为0。若数据分布是对称的，偏度 = 0。

     若偏度 > 0，分布为右偏，即分布有一条长尾在右；
     
     若偏度 < 0，分布为左偏，即分布有一条长尾在左。偏度的绝对值越大，说明分布的偏移程度越严重。


* <b> 峰度 </b>  正态分布的峰度为0。

    当峰度 > 0，它相比于正态分布要更陡峭或尾部更厚。
    
    峰度系数 < 0, 它相比于正态分布更平缓或尾部更薄。

Normality - When we talk about normality what we mean is that the data should look like a normal distribution. This is important because several statistic tests rely on this (e.g. t-statistics). In this exercise we'll just check univariate normality for 'SalePrice' (which is a limited approach). Remember that univariate normality doesn't ensure multivariate normality (which is what we would like to have), but it helps. Another detail to take into account is that in big samples (>200 observations) normality is not such an issue.

However, if we solve normality, we avoid a lot of other problems (e.g. heteroscedacity) so that's the main reason why we are doing this analysis.

正态分布

In [ ]:
train_df['SalePrice'] = np.log1p(train_df['SalePrice'])
sns.distplot(train_df['SalePrice'], color='blue')


In [ ]:
print (train_df['SalePrice'].skew())
train_df['SalePrice'].kurt()

### 【转化性】
Label Encoding 一些可以包含信息的类别特征

**表达类别特征之间的高低顺序Excellent > Good > bad** 

每单位变化跟目标变量并没有关系！

In [ ]:
from sklearn.preprocessing import LabelEncoder
# categories = [i for i in train_df.columns if i not in cols]
# process columns, apply LabelEncoder to categorical features

categories = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold', 'BedroomAbvGr']

for c in categories:
    lbl = LabelEncoder() 
    train_df[c] = lbl.fit_transform(list(train_df[c].values))
    test_df[c] = lbl.fit_transform(list(test_df[c].values))


# shape        
print ('Shape all_data: {}'.format(train_df.shape))
print ('Shape all_data: {}'.format(test_df.shape))


In [ ]:
numeric_features = list(train_df.dtypes[train_df.dtypes != 'object'].index)
numeric_features.remove('SalePrice')
len(numeric_features)

#### 转化后所有特征标准化

In [ ]:
# box-cox变换
from scipy.special import boxcox1p
# skewed_features = list(skewness)
lam = 0.15
for feature in numeric_features:
    #all_data[feat] += 1
    train_df[feature] = boxcox1p(train_df[feature], lam)
    test_df[feature] = boxcox1p(test_df[feature], lam)
    
#all_data[skewed_features] = np.log1p(all_data[skewed_features])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
scaler.fit(train_df[numeric_features])
train_df[numeric_features] = scaler.transform(train_df[numeric_features])
test_df[numeric_features] = scaler.transform(test_df[numeric_features])

# scaler = MinMaxScaler()
# scaler.fit(train_df[numeric_features])
# train_df[numeric_features] = scaler.transform(train_df[numeric_features])
# test_df[numeric_features] = scaler.transform(test_df[numeric_features])

In [ ]:
category_columns = train_df.dtypes[train_df.dtypes == 'object'].index

In [ ]:
for i in category_columns:
    if len(train_df[i].value_counts().index) <= 2:
        print  ("Train\n" +  i)
        print  (train_df[i].value_counts())
        
    if len(test_df[i].value_counts().index) <= 2:
        print ("Test\n" + i)
        print (test_df[i].value_counts())

In [ ]:
drop_columns = 'Utilities'
train_df = train_df.drop(drop_columns, axis=1)
test_df = test_df.drop(drop_columns, axis=1)

#### 转化性

In [ ]:
features_train = train_df.drop(['SalePrice'], axis=1)
labels_train = train_df['SalePrice']
features_test = test_df

#### 独热编码其他与顺序无关的特征

In [ ]:
features_train = pd.get_dummies(features_train)
features_test = pd.get_dummies(test_df)

missing_cols = set(features_train.columns) - set(features_test.columns)
for column in missing_cols:
    features_test[column] = 0
    
# 保证测试集columns的顺序同训练集columns相同，特别重要！！！！！！
features_test = features_test[features_train.columns]

In [ ]:
print (features_train.columns)

### 分割训练和测试数据

In [ ]:
# from sklearn.model_selection import train_test_split

# # 分割features_train 和 labels_train, 测试集大小 = 20%，状态：随机，可复现
# # 顺序：测试特征，训练特征，测试目标，训练目标

# X_train, X_test, y_train, y_test = train_test_split(features_train, labels_train, test_size = 0.2, random_state = 42)

# 输出数量观察

X_train = features_train
y_train = labels_train
print (len(X_train))

# 模型实现与融合

## 分析问题，确定模型

问题为回归问题，可用模型：线性回归，决策树（C&RT决策树），随机森林，GBDT

## 首先用简单的模型进行试验，观察评分

回归模型评分指标包括：

1、SSE误差平方和， RMSE

2、R-square（决定系数）

In [ ]:
# # 导入算法模型和评分标准 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import fbeta_score, make_scorer, r2_score ,mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# def rmsle_cv(model, train, value):
#     kf = KFold(5, shuffle=True, random_state=42).get_n_splits(train.values)
#     rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
#     return rmse.mean()

# # for key_num in range(10,25,3):

# #     cols_model = cols[:key_num]
# #     drop_columns = [i for i in corrmat.columns if i not in cols_model]
# #     print drop_columns , "\n"
# #     X_train_model = X_train.drop(drop_columns, axis=1)

# #         # 初始化,确定随机状态，可复现
# #     reg1 = DecisionTreeRegressor(random_state = 42)
# #     reg2 = LinearRegression()
# #     reg3 = RandomForestRegressor(random_state = 42)
# #     reg4 = XGBRegressor()
# #     reg5 = Lasso(alpha=0.001, random_state= 42)
# #     reg6 = SVR(kernel = 'rbf')
# #     reg7 = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3)

# #     # 建立字典，收集学习器的效果
# #     # 学习，收集预测得分
# #     results = {}
# #     for reg in [reg1, reg2, reg3, reg4, reg5, reg6, reg7]:
# #         # 回归器的名称
# #         reg_name = reg.__class__.__name__
# # #         reg.fit(X_train_model, y_train)
# # #         pred_test = reg.predict(X_test_model)
# # #         results[reg_name] = rmse(y_test, pred_test)
# #         results[reg_name] = rmsle_cv(reg, X_train_model, y_train)
# #     print key_num, "---", results
# #     print '\n'

In [ ]:
cols = cols[:22] 
drop_columns = [i for i in corrmat.columns if i not in cols]

print (drop_columns , "\n")
X_train = X_train.drop(drop_columns, axis=1)
print (len(X_train.columns))

In [ ]:
features_test = features_test.drop(drop_columns, axis=1)

## 网格搜索调参

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer, r2_score ,mean_squared_error

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

### 随机森林

In [ ]:
# 模型：RandomForest
# 导入Grid
from sklearn.model_selection import GridSearchCV

# 初始化回归模型
reg = RandomForestRegressor(random_state=42)

# 确定参数列表
parameters = {
    'max_leaf_nodes': range(20, 100 ,10)
}

# 确定评分标准
scorer = 'neg_mean_squared_error'

# 回归模型使用网格搜索
grid_reg = GridSearchCV(reg, parameters, scoring = scorer)

# 训练
grid_reg.fit(X_train, y_train)

# grid_reg.cv_results_

# 获得最佳拟合回归器
best_reg_rf = grid_reg.best_estimator_

In [ ]:
# print grid_reg.best_estimator_
print (grid_reg.best_estimator_)
pred_rf = best_reg_rf.predict(X_train)
rmsle(pred_rf, y_train)

### Lasso回归

一般来说，对于高维的特征数据，尤其线性关系是稀疏的，我们会采用Lasso回归。或者是要在一堆特征里面找出主要的特征，那么Lasso回归更是首选了。但是Lasso类需要自己对α调优，所以不是Lasso回归的首选，一般用到的是下一节要讲的LassoCV类。

https://www.cnblogs.com/pinard/p/6026343.html

In [ ]:
# 模型：线性回归
from sklearn.model_selection import GridSearchCV

# 初始化回归模型
reg = Lasso( alpha=0.001, random_state= 42)

# 确定参数列表
parameters = {
    'normalize': [True, False],
    'alpha': [0, 0.0001, 0.0005, 0.001]
}

# 确定评分标准
scorer = 'neg_mean_squared_error'

# 回归模型使用网格搜索
grid_reg = GridSearchCV(reg, parameters, scoring = scorer)

# 训练
grid_reg.fit(X_train, y_train)

# 获得最佳拟合回归器
best_reg_lasso = grid_reg.best_estimator_

In [ ]:
print (grid_reg.best_estimator_)
pred_lasso = best_reg_lasso.predict(X_train)
rmsle(pred_lasso, y_train)

### ElasticNet

弹性网络，结合L1和L2正则化

In [ ]:
reg = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state = 42)

parameters = {
    'alpha':[0, 0.0001, 0.0005, 0.001],
    'l1_ratio':np.arange(0, 1, 0.1)
}

scorer = 'neg_mean_squared_error'

grid_reg = GridSearchCV(reg, parameters, scoring= scorer)

grid_reg.fit(X_train, y_train)

best_elasticNet_reg =  grid_reg.best_estimator_

In [ ]:
print (grid_reg.best_estimator_)
pred_elasticNet_reg = best_elasticNet_reg.predict(X_train)
rmsle(pred_elasticNet_reg, y_train)


### SVR_rbf 

In [ ]:
from sklearn.svm import SVR

reg = SVR(kernel = 'rbf')

parameters = {
    'C': np.arange(1.1,2,0.1),
    'gamma':np.arange(0,0.1,0.01)
}

scorer = 'neg_mean_squared_error'

grid_reg = GridSearchCV(reg, parameters, scoring = scorer)

grid_reg.fit(X_train, y_train)

best_reg_SVR = grid_reg.best_estimator_

In [ ]:
print (grid_reg.best_estimator_)
pred_lasso = best_reg_lasso.predict(X_train)
rmsle(pred_lasso, y_train)

### Xgboost

### 1、确定学习速率和tree_based 参数调优的估计器数目

In [ ]:
# # 确定最佳决策树的数量, xgboost中的cv函数来确定最佳的决策树数量, 提高Xgboost调参速度
# # 后边有 early_stopping_rounds 个rmse没有下降的就停止
# # 原话：
# # Activates early stopping. CV error needs to decrease at least
# #        every <early_stopping_rounds> round(s) to continue.
# #        Last entry in evaluation history is the one from best iteration.

# import xgboost as xgb
# from xgboost.sklearn import XGBRegressor

# def modelfit(alg, X_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
#     if useTrainCV:
#         xgb_param = alg.get_xgb_params()
#         xgtrain = xgb.DMatrix(X_train.values, label= y_train.values)
#         cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
#             metrics='rmse', early_stopping_rounds = early_stopping_rounds)
#         alg.set_params(n_estimators=cvresult.shape[0])
#         print cvresult

#     #Fit the algorithm on the data
#     alg.fit(X_train, y_train,eval_metric='rmse')

#     #Predict training set:
#     dtrain_predictions = alg.predict(X_train)

#     # 模型训练报告
#     print "\nModel Report"
#     print "RMSE Score (Train): %f" % rmse(y_train, dtrain_predictions)
    
#     # 特征重要性
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)[:10,]
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')

In [ ]:
# 模型：Xgboost
from sklearn.model_selection import GridSearchCV

best_reg_xgb = XGBRegressor(learning_rate= 0.01, n_estimators = 5000, 
                    max_depth= 4, min_child_weight = 1.5, gamma = 0.1, 
                   subsample = 0.7, colsample_bytree = 0.6, 
                   seed = 27)

# modelfit(reg, X_train, y_train)

In [ ]:
best_reg_xgb.fit(X_train, y_train)
pred_y_XGB = best_reg_xgb.predict(X_train)
print (rmsle(pred_y_XGB, y_train))

学习速率为0.1时，理想的决策树数目为115个

In [ ]:
# rmsle_cv(best_reg_xgb, X_train, y_train)

### 2、调整Xgboost相关参数

#### max_depth 和 min_weight 参数调优

max_depth 树的最大深度

min_child_weight 最小叶子节点样本权重和：




In [ ]:
# param_test1 = {
#     'max_depth': range(3,10,2),
#     'min_child_weight': range(1,6,2)
# }

# scorer = make_scorer(rmse)

# # 负均方误差
# grid_reg1 = GridSearchCV(estimator=XGBRegressor(learning_rate=0.01, n_estimators=1605, 
#                                                 max_depth=5, min_child_weight = 1,gamma = 0,
#                                                 subsample = 0.8, colsample_bytree = 0.8, seed = 27), 
#                          param_grid = param_test1, scoring = 'neg_mean_squared_error')
# grid_reg1.fit(X_train, y_train)
# grid_reg1.grid_scores_, grid_reg1.best_params_, grid_reg1.best_score_


理想max_depth = 5， min_child_weight = 3，值附近细化调整

In [ ]:
# param_test1 = {
#     'max_depth': [4, 5, 6],
#     'min_child_weight': np.arange(1.0, 4.0, 0.5)
# }

# scorer = make_scorer(rmse)

# # 负均方误差
# grid_reg1 = GridSearchCV(estimator=XGBRegressor(learning_rate=0.1, n_estimators=115, 
#                                                 max_depth=5, min_child_weight = 1,gamma = 0,
#                                                 subsample = 0.8, colsample_bytree = 0.8, seed = 27), 
#                          param_grid = param_test1, scoring = 'neg_mean_squared_error')
# grid_reg1.fit(X_train, y_train)
# grid_reg1.grid_scores_, grid_reg1.best_params_, grid_reg1.best_score_

'max_depth'理想值 4, 'min_child_weight' 理想值 1.5 交叉验证得分：0.016

#### gamma参数调优

节点分裂所需的最小损失函数下降值

In [ ]:
# param_test3 = {
#     'gamma' : [i/10.0 for i in range(0,5)]
# }


# grid_reg1 = GridSearchCV(estimator=XGBRegressor(learning_rate=0.1, n_estimators=115, 
#                                                 max_depth=4, min_child_weight = 1.5,gamma = 0,
#                                                 subsample = 0.8, colsample_bytree = 0.8, seed = 27), 
#                          param_grid = param_test3, scoring = 'neg_mean_squared_error')
# grid_reg1.fit(X_train, y_train)
# grid_reg1.grid_scores_, grid_reg1.best_params_, grid_reg1.best_score_

学习速率为0.1的情况下，理想决策树115， 'max_depth'理想值 4, 'min_child_weight' 理想值 1.5， gamma为0

#### 调整subsample 和 colsample_bytree 参数

样本抽样和列抽样的比重

In [ ]:
# param_test4 = {
#     'subsample':np.arange(0.5, 1.0 ,0.05),
#     'colsample_bytree':np.arange(0.5, 1.0, 0.05)
# }


# grid_reg1 = GridSearchCV(estimator=XGBRegressor(learning_rate=0.1, n_estimators=115, 
#                                                 max_depth=4, min_child_weight = 1.5,gamma = 0,
#                                                 subsample = 0.8, colsample_bytree = 0.8, seed = 27), 
#                          param_grid = param_test4, scoring = 'neg_mean_squared_error')
# grid_reg1.fit(X_train, y_train)
# grid_reg1.grid_scores_, grid_reg1.best_params_, grid_reg1.best_score_

学习速率为0.1的情况下，理想决策树115， 'max_depth'理想值 4, 'min_child_weight' 理想值 1.5， gamma为0, 

colsample_bytree为0.6， subsample为0.7，MSE为0.0156

#### 3、正则化参数调优

In [ ]:
# param_test5 = {
#     "reg_alpha":np.arange(0.0, 1.1, 0.1),
#     "reg_lambda":np.arange(0.0, 1.1, 0.1)
# }

# grid_reg1 = GridSearchCV(estimator=XGBRegressor(learning_rate=0.1, n_estimators=115, 
#                                                 max_depth=4, min_child_weight = 1.5,gamma = 0,
#                                                 subsample = 0.7, colsample_bytree = 0.6, seed = 27), 
#                          param_grid = param_test5, scoring = 'neg_mean_squared_error')
# grid_reg1.fit(X_train, y_train)
# grid_reg1.grid_scores_, grid_reg1.best_params_, grid_reg1.best_score_

学习速率为0.1的情况下，理想决策树115， 

'max_depth'理想值 4, 'min_child_weight' 理想值 1.5， gamma为0,

colsample_bytree为0.6， subsample为0.7

alpha为0.1，lambda为0

MSE为0.0154

#### 4、降低学习速率，确定理想参数

使用较低的学习速率，更多的决策树

In [ ]:
# reg2 = XGBRegressor(learning_rate=0.1, n_estimators= 1000, 
#                          max_depth=4, min_child_weight = 1.5,gamma = 0,
#                         subsample = 0.7, colsample_bytree = 0.6, seed = 27)

# modelfit(reg2, X_train, y_train )

In [ ]:
# pred_y_test = reg2.predict(X_test)
# rmse(pred_y_test, y_test)

学习速率为0.01的情况下，理想决策树5000， 

'max_depth'理想值 4, 'min_child_weight' 理想值 1.5， gamma为0,

colsample_bytree为0.6， subsample为0.7

alpha为0.1，lambda为0

MSE为

1、仅靠参数的调整和模型的小幅优化，想要让模型的表现有个大幅度提升是不可能的

2、要想让模型的表现有一个质的飞跃，需要依靠其他的手段：

    1、特征工程(feature egineering) 
    
    2、模型组合(stacking)

## 模型融合

In [ ]:
from mlxtend.regressor import StackingRegressor

# metal_reg = Lasso(alpha= 0.0, random_state=42)
metal_reg = SVR(kernel= 'rbf', C = 20)

stregr = StackingRegressor(regressors = [ best_reg_lasso, best_elasticNet_reg], meta_regressor = metal_reg)

# params = {'meta-lasso__alpha':[0.1, 1.0, 10.0] }

# grid = GridSearchCV(estimator=stregr,
#                     param_grid=params,
#                     cv=5,
#                     refit=True)

# grid.fit(X_train, y_train)
# for params, mean_score, scores in grid.grid_scores_:
#     print("%0.3f +/- %0.2f %r"
#         % (mean_score, scores.std() / 2.0, params))


# rmsle_cv(stregr, X_train, y_train).mean()

In [ ]:
stregr.fit(X_train, y_train)
stacked_y_pred  = stregr.predict(X_train)
rmsle(y_train, stacked_y_pred)

### 学习曲线

使用Sklearn中学习曲线函数：http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

In [ ]:
# print(__doc__)

# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from sklearn.datasets import load_digits
# from sklearn.model_selection import learning_curve
# from sklearn.model_selection import ShuffleSplit


# def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
#                         n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
#     """
#     Generate a simple plot of the test and training learning curve.

#     Parameters
#     ----------
#     estimator : object type that implements the "fit" and "predict" methods
#         An object of that type which is cloned for each validation.

#     title : string
#         Title for the chart.

#     X : array-like, shape (n_samples, n_features)
#         Training vector, where n_samples is the number of samples and
#         n_features is the number of features.

#     y : array-like, shape (n_samples) or (n_samples, n_features), optional
#         Target relative to X for classification or regression;
#         None for unsupervised learning.

#     ylim : tuple, shape (ymin, ymax), optional
#         Defines minimum and maximum yvalues plotted.

#     cv : int, cross-validation generator or an iterable, optional
#         Determines the cross-validation splitting strategy.
#         Possible inputs for cv are:
#           - None, to use the default 3-fold cross-validation,
#           - integer, to specify the number of folds.
#           - An object to be used as a cross-validation generator.
#           - An iterable yielding train/test splits.

#         For integer/None inputs, if ``y`` is binary or multiclass,
#         :class:`StratifiedKFold` used. If the estimator is not a classifier
#         or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

#         Refer :ref:`User Guide <cross_validation>` for the various
#         cross-validators that can be used here.

#     n_jobs : integer, optional
#         Number of jobs to run in parallel (default 1).
#     """
#     plt.figure()
#     plt.title(title)
#     if ylim is not None:
#         plt.ylim(*ylim)
#     plt.xlabel("Training examples")
#     plt.ylabel("Score")
#     train_sizes, train_scores, test_scores = learning_curve(
#         estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
#     train_scores_mean = np.mean(train_scores, axis=1)
#     train_scores_std = np.std(train_scores, axis=1)
#     test_scores_mean = np.mean(test_scores, axis=1)
#     test_scores_std = np.std(test_scores, axis=1)
#     plt.grid()

#     plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
#                      train_scores_mean + train_scores_std, alpha=0.1,
#                      color="r")
#     plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
#                      test_scores_mean + test_scores_std, alpha=0.1, color="g")
#     plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
#              label="Training score")
#     plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
#              label="Cross-validation score")

#     plt.legend(loc="best")
#     return plt

In [ ]:
# title = "Learning Curves (Xgboost)"
# # Cross validation with 100 iterations to get smoother mean test and train
# # score curves, each time with 20% data randomly selected as a validation set.
# cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

# estimator = XGBRegressor()
# plot_learning_curve(estimator, title, features_train, labels_train, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

# plt.show()

# 预测目标数据

In [ ]:
pred = np.expm1(stregr.predict(features_test) * 0.5 + best_reg_xgb.predict(features_test) * 0.5)
pred

# 输出结果

In [ ]:
# 增加索引，列名，构建DataFrame，符合输出数据格式

test_df = pd.read_csv('../input/test.csv')
predict_df = pd.DataFrame({'Id': test_df['Id'], 'SalePrice': pred})

# DataFrame设定index
predict_df = predict_df.set_index('Id')

# 重命名DataFrame的列，列名 = 字典{原：替换后}
# predict_df.rename(columns = {predict_df.columns[0]: 'Id'}, inplace=True)

predict_df

In [ ]:
predict_df.to_csv('Submission.csv')

# 待优化策略

ROUND 1：

I：基础机器学习框架搭建,采用Random Forest 0.18129 

II：Xgboost + 所有特征: 0.14339 ⬆️（可解释性差，容易过拟合）

III：只取相关性强数值特征 ：0.14870 ⬇️

IV：相关性强数值特征 + Label Coder ：0.14628 ⬇️

V: 部分数值特征转换为类别特征：Msubclass等：0.13902 ⬆️

VI：部分类别特征Label coder有序；部分独热编码：0.13814 ⬆️

VII：处理倾斜特征和目标变量：0.13806 ⬆️
 
ROUND2：

I：XGboost调参 0.1325 -> 0.131 -> 0.12907  ⬆️

II：多种算法stacking：0.12818 ⬆️

III：前10有效特征，融合stacking和xgb：0.12584 ⬆️

IIII：过拟合较严重，简化模型，stacking去除RF：0.12138 ⬆️


待优化点有：

1、评分标准更改为Kaggle标准 ✅

2、特征分析，创造新特征 ✅

特征较多：74，如何选取以及探索恰当的特征？

3、探索性分析，确定关键特征 ✅

【探索性数据分析】

4、Xgboost学习，调参  ✅

5、集成方法 stacking   ✅

6、模型有些过拟合，试图减少特征的数量 ✅


待解决问题有：

1、怎么判断模型存在过拟合？✅

学习曲线，Learning curve

2、线性回归为什么有负值 ✅

在做类别数据 -> 数值转化时，训练集同测试集列名存在区别。只新增了测试集缺失的列，未调整列的顺序。
注意：test = test[train.columns]

3、XGboost调参技巧 ✅